# BB84 Quantum Key Distribution (QKD) Protocol (with eavesdropping)

This notebook is a _demonstration_ of the BB84 Protocol for QKD using Qiskit. 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
The first three sections of the paper are readable and should give you all the necessary information required. 


![QKD Setup](https://raw.githubusercontent.com/deadbeatfour/quantum-computing-course/master/img/qkd_eavesdropping.png)


[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

The other encodes the state:

$0  \rightarrow|0\rangle$ or $|+\rangle $ 

$1  \rightarrow|1\rangle$  or  $|-\rangle $ 

Bob and Oscar also generate a binary string each using the same convention to choose a basis for measurement


In [28]:
num_qubits = 32

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)
oscar_basis = np.random.randint(2, size=num_qubits)

print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
print(f"Alice's Bases:\t {np.array2string(alice_basis, separator='')}")
print(f"Oscar's Bases:\t {np.array2string(oscar_basis, separator='')}")
print(f"Bob's Bases:\t {np.array2string(bob_basis, separator='')}")

Alice's State:	 [11010011010110001001001011010011]
Alice's Bases:	 [11010100100110111011110100001001]
Oscar's Bases:	 [01011001111011010101100101111100]
Bob's Bases:	 [01011001011000101101001000110001]


## Creating the circuit

Based on the following results:

$X|0\rangle = |1\rangle$

$H|0\rangle = |+\rangle$

$ HX|0\rangle = |-\rangle$


Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. However, Oscar **intercepts** the qubits and measures them by choosing a basis as per his generated random binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a measurement on the computational basis. 

5. Oscar now prepares another set of qubits according to his measurements and the bases he chose. He then **re-sends** these qubits to Bob.

4. Bob measures the qubits according to his binary string. Bob also measures using the same method as Eve.

Since this can be seen as two BB84 steps in tandem, we can use the framework that we developed earlier.

In [29]:
def make_bb84_circ(enc_state, enc_basis, meas_basis):
    '''
    enc_state: array of 0s and 1s denoting the state to be encoded
    enc_basis: array of 0s and 1s denoting the basis to be used for encoding
                0 -> Computational Basis
                1 -> Hadamard Basis
    meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                0 -> Computational Basis
                1 -> Hadamard Basis
    '''
    num_qubits = len(enc_state)
    
    bb84_circ = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    for index in range(len(enc_basis)):
        if enc_state[index] == 1:
            bb84_circ.x(index)
        if enc_basis[index] == 1:
            bb84_circ.h(index)
    bb84_circ.barrier()  

    # Receiver measures the received qubits
    for index in range(len(meas_basis)):
        if meas_basis[index] == 1:
            bb84_circ.h(index)

    bb84_circ.barrier()        
    bb84_circ.measure_all()
    
    return bb84_circ


## Simulating intercepted BB84
The 'intercept and re-send' attack can be simulated by thinking of the whole process being broken up into two parts. The first part can be thought of as the BB84 protocol happening between Alice and Oscar, and the second part between Oscar and Bob. However, we have to know the result from the first part to create the circuit for the second part. We will do this below.

In [30]:
bb84_AO = make_bb84_circ(alice_state, alice_basis, oscar_basis)
oscar_result = execute(bb84_AO.reverse_bits(),
            backend=QasmSimulator(),
            shots=1).result().get_counts().most_frequent()
print(f"Oscar's results:\t {oscar_result}")
# Converting string to array
oscar_state = np.array(list(oscar_result), dtype=int)
print(f"Oscar's State:\t\t{np.array2string(oscar_state, separator='')}")

Oscar's results:	 01011111001010101111011011010010
Oscar's State:		[01011111001010101111011011010010]


In [31]:
bb84_OB = make_bb84_circ(oscar_state, oscar_basis, bob_basis)
temp_key = execute(bb84_OB.reverse_bits(),
                   backend=QasmSimulator(),
                   shots=1).result().get_counts().most_frequent()

## Creating the key

Alice and Bob only keep the bits where their bases match. Oscar also keeps only these bits from his measurements.

In [34]:
alice_key = ''
bob_key = ''
oscar_key = ''
for i in range(num_qubits):
    if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
        alice_key += str(alice_state[i])
        bob_key += str(temp_key[i])
        oscar_key += str(oscar_result[i])
print(f"The length of the key is {len(bob_key)}")
print(f"Alice's key contains\t {(alice_key).count('0')} zeroes and {(alice_key).count('1')} ones")
print(f"Bob's key contains\t {(bob_key).count('0')} zeroes and {(bob_key).count('1')} ones")
print(f"Oscar's key contains\t {(oscar_key).count('0')} zeroes and {(oscar_key).count('1')} ones")
print(f"Alice's Key:\t {alice_key}")
print(f"Bob's Key:\t {bob_key}")
print(f"Oscar's Key:\t {oscar_key}")

The length of the key is 13
Alice's key contains	 4 zeroes and 9 ones
Bob's key contains	 5 zeroes and 8 ones
Oscar's key contains	 4 zeroes and 9 ones
Alice's Key:	 1011001111011
Bob's Key:	 1011101110010
Oscar's Key:	 1011011111010
